In [ ]:
#import library to generate sufficiently random initial key value, have to pip install pycryptodome
from Crypto.Random import get_random_bytes

#S-box provided in lesson slides for encryption 
sbox = {'0000':'1001', '0001': '0100', '0010': '1010', '0011': '1011', '0100': '1101',
         '0101': '0001', '0110': '1000', '0111': '0101', '1000': '0110', '1001': '0010',
          '1010': '0000', '1011': '0011', '1100': '1100', '1101': '1110', '1110': '1111',
          '1111': '0111'}

#inverse S-box for decryption
inversesbox = {'1001': '0000', '0100': '0001', '1010': '0010', '1011': '0011', '1101': '0100',
               '0001': '0101', '1000': '0110', '0101': '0111', '0110': '1000', '0010': '1001',
               '0000':'1010', '0011': '1011', '1100':'1100', '1110': '1101', '1111': '1110',
               '0111':'1111'}

#round constants for key expansion: binary representations of x**(i+2)/x**4+x+1 + 4 0's to make 8 bits
rconstants =  ['10000000', '00110000', '11000000', '00110000', '0001000', '00100000', '01000000',
               '10000000', '00110000', '11000000', '00110000', '00010000', '00100000', '01000000',
               '10000000']

#list for storing user-provided plaintexts
#plaintexts = []
testing_plaintexts = ['lions', 'tigers', 'bears', 'walruses', 'deer',
                       'giraffes', 'llamas', 'ostriches', 'wolves', 'whales']

#converted plaintexts to binary strings
#plaintexts_binary = []
testing_plaintexts_binary = []

#max number of plaintexts to store
numofplaintexts = 10

#take 10 user inputs and store them as strings and binary strings
#for i in range(numofplaintexts):
#    userinput = input("Please enter the plaintext: ")
#    userinputbinary = ''.join(format(byte, '08b') for byte in bytearray(userinput, encoding='utf-8'))
#    plaintexts.append(userinput)
#    plaintexts_binary.append(userinputbinary)

#to fill a test list of binary values to encrypt
for i in range(len(testing_plaintexts)):
    testinput = ''.join(format(byte, '08b') for byte in bytearray(testing_plaintexts[i], encoding='utf-8'))
    testing_plaintexts_binary.append(testinput)


key0 = get_random_bytes(2) # 2 bytes * 8 = 16 bits 
binaryKey0 = ''.join(format(byte, '08b') for byte in key0)
binaryKey1 = None
binaryKey2 = None

#performs key expansion to derive key values for rounds 1 and 2 of SAES encryption
def expand_key(init_key_val, roundconstant):
    word0 = init_key_val[0:8]
    word1 = init_key_val[8:16]
    #breaking bytes into 4 bit "nibs"
    word0nib0, word0nib1, word1nib0, word1nib1 = word0[0:4], word0[4:8], word1[0:4], word1[4:8]
    
    #apply rotation
    rotatednibs = word1nib1 + word1nib0
    
    #apply substition from provided sbox 
    sub_rotnib1 = sbox[rotatednibs[:4]] + sbox[rotatednibs[4:]]
    
    #XOR the rotated and substituted word1 with rconstant0, pad with any necessary '0's
    xored_subrotword1 = bin(int(roundconstant, 2) ^ int(sub_rotnib1, 2))[2:].rjust(8, '0')
    
    #XOR result with word0 to get word2, pad with any necessary '0's
    word2 = bin(int(word0, 2) ^ int(xored_subrotword1, 2))[2:].rjust(8, '0')
    
    #XOR word2 with word1 to get word3, pad with any necessary '0's
    word3 = bin(int(word2, 2) ^ int(word1, 2))[2:].rjust(8, '0')
    
    return(word2 + word3)
    
binaryKey1 = expand_key(binaryKey0, rconstants[0])
binaryKey2 = expand_key(binaryKey1, rconstants[1])    
#print(binaryKey0 + '\n' + binaryKey1 + '\n' + binaryKey2)
#for i in range(len(testing_plaintexts_binary)):
#    print(testing_plaintexts_binary[i])
#    print(len(testing_plaintexts_binary[i]))

#carry out first step of SAES encryption - substitution, with input value a string of the binary digits representing plaintext inputs
def substitute_nibbles(plaintext_binary):
    nibbles = []
    subbed_nibbled_words = []
    
    #divide into 4 bit nibbles
    for i in range(len(plaintext_binary)):
        nibbles.append([testing_plaintexts_binary[i][j:j+4] for j in range(0, len(testing_plaintexts_binary[i]), 4)]) 
    
    #replace plaintext nibble with corresponding Sbox value
    for i in range(len(nibbles)):
        subbed_nibbles = []
        for j in range(len(nibbles[i])):
            subbed_nibbles.append(sbox[nibbles[i][j]]) 
        subbed_nibbled_words.append(subbed_nibbles)
    return(subbed_nibbled_words)

test_subbed_binary = substitute_nibbles(testing_plaintexts_binary)

#carry out second step of SAES encryption - shift rows, with input value a string of the binary digits representing plaintext inputs that have been substituted
def shift_rows(subbed_binary):
    #check for divisibility by 16, if not, pad with '0's
    for i in range(len(subbed_binary)):
        if (len(subbed_binary[i])*4) % 16 != 0:
            subbed_binary[i].extend(['0000', '0000'])
    #shift rows in a 2x2 matrix is equivalent to swapping every second and fourth nibbles 
    for i in range(len(subbed_binary)):
        for j in range(0, len(subbed_binary[i]), 4):
            temp_value = subbed_binary[i][j+1]
            subbed_binary[i][j+1] = subbed_binary[i][j+3]
            subbed_binary[i][j+3] = temp_value
        return(subbed_binary)
        
test_shifted_binary = shift_rows(test_subbed_binary)                    
    

def mix_columns():
    None

def encrypt_SAES():
    None

def decrypt_SAES():
    None